## Part VII.2 - Train DeepAR model and publish to model store

University of San Diego - MS Applied AI

AAI-540 Team 5

October 21, 2024

In [ ]:
# setup environment
%run 0-Environment_Setup.ipynb

In [ ]:
import json
from time import gmtime, strftime

In [ ]:
# configure model image and output path
image_name = sagemaker.image_uris.retrieve("forecasting-deepar", region)
s3_output_path = s3_dataset_path + "/output"

In [ ]:
# initialize estimator
estimator = sagemaker.estimator.Estimator(
    image_uri=image_name,
    sagemaker_session=sess,
    role=role,
    train_instance_count=1,
    train_instance_type="ml.m5.xlarge",
    base_job_name="store-sales-forecasting-deepar",
    output_path=s3_output_path,
)

In [ ]:
# define vanilla hyperparameters
hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
}

In [ ]:
# set hyperparameters to model
estimator.set_hyperparameters(**hyperparameters)

In [ ]:
# train the model
data_channels = {"train": "{}/train/".format(s3_dataset_path), "test": "{}/test/".format(s3_dataset_path)}
estimator.fit(inputs=data_channels, wait=True)